<a href="https://colab.research.google.com/github/Kang-Kim-Cross/Lotto-Crawl/blob/main/lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re
import json
from multiprocessing import Process, Queue
import math

In [2]:
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'

In [3]:
response = requests.get(url)
response.content

b'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="EUC-KR">\r\n<meta id="utitle" name="title" content="\xb5\xbf\xc7\xe0\xba\xb9\xb1\xc7">\r\n<meta id="desc" name="description" content="\xb5\xbf\xc7\xe0\xba\xb9\xb1\xc7 999\xc8\xb8 \xb4\xe7\xc3\xb7\xb9\xf8\xc8\xa3 1,3,9,14,18,28+34. 1\xb5\xee \xc3\xd1 16\xb8\xed, 1\xc0\xce\xb4\xe7 \xb4\xe7\xc3\xb7\xb1\xdd\xbe\xd7 1,513,274,790\xbf\xf8.">\r\n<title>\xb7\xce\xb6\xc76/45 - \xc8\xb8\xc2\xf7\xba\xb0 \xb4\xe7\xc3\xb7\xb9\xf8\xc8\xa3</title>\r\n<title>\xb5\xbf\xc7\xe0\xba\xb9\xb1\xc7</title>\r\n\r\n    \r\n    \r\n        \r\n\r\n\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<link rel="shortcut icon" href="/images/common/favicon.ico" type="image/x-icon">\r\n<link rel="icon" href="/images/common/favicon.ico" type="image/x-icon">\r\n<script type="text/javascript" src="/js/jquery-1.9.1.min.js"></script>\r\n<script type="text/javascript" src="/js/j

In [4]:
body = response.content.decode('euc-kr')
body

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="EUC-KR">\r\n<meta id="utitle" name="title" content="동행복권">\r\n<meta id="desc" name="description" content="동행복권 999회 당첨번호 1,3,9,14,18,28+34. 1등 총 16명, 1인당 당첨금액 1,513,274,790원.">\r\n<title>로또6/45 - 회차별 당첨번호</title>\r\n<title>동행복권</title>\r\n\r\n    \r\n    \r\n        \r\n\r\n\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<link rel="shortcut icon" href="/images/common/favicon.ico" type="image/x-icon">\r\n<link rel="icon" href="/images/common/favicon.ico" type="image/x-icon">\r\n<script type="text/javascript" src="/js/jquery-1.9.1.min.js"></script>\r\n<script type="text/javascript" src="/js/jquery-ui.js"></script>\r\n<script type="text/javascript" src="/js/common.js" charset="utf-8"></script>\r\n<script type="text/javascript">\r\n\r\nfn_g_init_message("");\r\n\r\nvar gameUserId = "";\r\n\r\nfunction goGame(){\r\n\tvar userId = "";\r\n\t\r\n

In [5]:
pattern = re.compile('동행복권\s([\d]{1,4})회')

In [6]:
result = re.findall(pattern, body)
result

['999']

In [7]:
latestDrwNo = int(result[0])
latestDrwNo

999

In [8]:
apiUrl = f'https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo='

In [9]:
apiResponse = requests.get(f'{apiUrl}{latestDrwNo}')
apiResponse.content

b'{"totSellamnt":102581351000,"returnValue":"success","drwNoDate":"2022-01-22","firstWinamnt":1513274790,"drwtNo6":28,"drwtNo4":14,"firstPrzwnerCo":16,"drwtNo5":18,"bnusNo":34,"firstAccumamnt":24212396640,"drwNo":999,"drwtNo2":3,"drwtNo3":9,"drwtNo1":1}'

In [10]:
apiResponse.encoding

'euc-kr'

In [11]:
apiResponse.content.decode(apiResponse.encoding)

'{"totSellamnt":102581351000,"returnValue":"success","drwNoDate":"2022-01-22","firstWinamnt":1513274790,"drwtNo6":28,"drwtNo4":14,"firstPrzwnerCo":16,"drwtNo5":18,"bnusNo":34,"firstAccumamnt":24212396640,"drwNo":999,"drwtNo2":3,"drwtNo3":9,"drwtNo1":1}'

In [12]:
json.loads(apiResponse.content.decode(apiResponse.encoding))

{'bnusNo': 34,
 'drwNo': 999,
 'drwNoDate': '2022-01-22',
 'drwtNo1': 1,
 'drwtNo2': 3,
 'drwtNo3': 9,
 'drwtNo4': 14,
 'drwtNo5': 18,
 'drwtNo6': 28,
 'firstAccumamnt': 24212396640,
 'firstPrzwnerCo': 16,
 'firstWinamnt': 1513274790,
 'returnValue': 'success',
 'totSellamnt': 102581351000}

In [13]:
def loadLottoData(drwNo):
  res = requests.get(f'https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={drwNo}')
  return json.loads(res.content.decode(res.encoding))

In [14]:
lottoData = Queue()

In [15]:
def lottoDataRangeLoader(start, end, queueData):
  dataList = []
  for drwNo in range(start, end):
    dataList.append(loadLottoData(drwNo=drwNo))
  queueData.put(dataList)

In [16]:
for idx in range(1, 10):
  print(idx)

1
2
3
4
5
6
7
8
9


In [17]:
processList = []

In [18]:
processNum = 100

In [19]:
for pid in range(math.ceil(latestDrwNo / processNum)):
  start = pid * processNum + 1
  if start + processNum > latestDrwNo:
    end = latestDrwNo + 1
  else:
    end = start + processNum
  print(pid, start, end)
  # proc = Process(target=lottoDataRangeLoader, args=())

0 1 101
1 101 201
2 201 301
3 301 401
4 401 501
5 501 601
6 601 701
7 701 801
8 801 901
9 901 1000


In [20]:
int(latestDrwNo / processNum), latestDrwNo % int(latestDrwNo / processNum)

(9, 0)

In [21]:
reqNum = 10
processList = []
lottoData = Queue()

In [22]:
for pid in range(math.ceil(1001 / reqNum)):
  start = pid * reqNum + 1
  if start + reqNum > 1001:
    end = 1001 + 1
  else:
    end = start + reqNum
  print(pid, start, end)

0 1 11
1 11 21
2 21 31
3 31 41
4 41 51
5 51 61
6 61 71
7 71 81
8 81 91
9 91 101
10 101 111
11 111 121
12 121 131
13 131 141
14 141 151
15 151 161
16 161 171
17 171 181
18 181 191
19 191 201
20 201 211
21 211 221
22 221 231
23 231 241
24 241 251
25 251 261
26 261 271
27 271 281
28 281 291
29 291 301
30 301 311
31 311 321
32 321 331
33 331 341
34 341 351
35 351 361
36 361 371
37 371 381
38 381 391
39 391 401
40 401 411
41 411 421
42 421 431
43 431 441
44 441 451
45 451 461
46 461 471
47 471 481
48 481 491
49 491 501
50 501 511
51 511 521
52 521 531
53 531 541
54 541 551
55 551 561
56 561 571
57 571 581
58 581 591
59 591 601
60 601 611
61 611 621
62 621 631
63 631 641
64 641 651
65 651 661
66 661 671
67 671 681
68 681 691
69 691 701
70 701 711
71 711 721
72 721 731
73 731 741
74 741 751
75 751 761
76 761 771
77 771 781
78 781 791
79 791 801
80 801 811
81 811 821
82 821 831
83 831 841
84 841 851
85 851 861
86 861 871
87 871 881
88 881 891
89 891 901
90 901 911
91 911 921
92 921 931
93 931 

In [23]:
for pid in range(math.ceil(latestDrwNo / reqNum)):
  start = pid * reqNum + 1
  if start + reqNum > latestDrwNo:
    end = latestDrwNo + 1
  else:
    end = start + reqNum
  proc = Process(target=lottoDataRangeLoader, args=(start, end, lottoData))
  proc.start()
  processList.append(proc)

In [ ]:
for proc in processList:
  proc.join()